In [1]:
from nose.tools import assert_equal

In [47]:
MOVIES_PATH = "/Users/Constance/ml-latest-small/movies.csv"
RATINGS_PATH = "/Users/Constance/ml-latest-small/ratings.csv"

!ln -sf $MOVIES_PATH movies.csv
!ln -sf $RATINGS_PATH ratings.csv

In [48]:
%%writefile movies.sql

-- Create the movies table from our familiar movies.csv file.
DROP TABLE IF EXISTS movies;

CREATE TABLE movies(
    movieId INT,
    title TEXT,
    genres TEXT
);

.separator ,
.import movies.csv movies

-- The following line removes the header.
DELETE FROM movies WHERE movieId='movieId';

-- Display the first 5 rows.
SELECT * FROM movies LIMIT 5;

Overwriting movies.sql


In [49]:
!sqlite3 problem3.db < movies.sql

1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [50]:
%%writefile ratings.sql

-- Create the ratings table from ratings.csv.
DROP TABLE IF EXISTS ratings;

CREATE TABLE ratings(
    userId INT,
    movieId INT,
    rating REAL,
    timestamp INT
);

.separator ,
.import ratings.csv ratings

-- The following line removes the header.
DELETE FROM ratings WHERE userId='userId';

-- Display the first 5 rows.
SELECT * FROM ratings LIMIT 5;

Overwriting ratings.sql


In [51]:
!sqlite3 problem3.db < ratings.sql

1,31,2.5,1260759144
1,1029,3.0,1260759179
1,1061,3.0,1260759182
1,1129,2.0,1260759185
1,1172,4.0,1260759205


### 1. Create the validRatings table

In [20]:
%%writefile ratings_operations.sql
DROP TABLE IF EXISTS validRatings;
CREATE TABLE validRatings AS
  SELECT movieId, AVG(rating) AS averageRating, COUNT(rating) AS ratingCounts
    FROM ratings
    GROUP BY movieId
    HAVING COUNT(rating)>30;

Overwriting ratings_operations.sql


In [21]:
!sqlite3 problem3.db < ratings_operations.sql

In [22]:
!sqlite3 problem3.db "SELECT * FROM validRatings LIMIT 5;"

1|3.87246963562753|247
2|3.4018691588785|107
3|3.16101694915254|59
5|3.26785714285714|56
6|3.88461538461538|104


In [17]:
# sql assertion tests

table_exists = !sqlite3 problem3.db "SELECT name FROM sqlite_master WHERE type='table' and name='validRatings';"
assert_equal(table_exists.s, "validRatings", msg="There's no table named \"validRatings\" in the database.")

table_info = !sqlite3 problem3.db "PRAGMA table_info(validRatings);"
columns = [t.split("|")[1] for t in table_info]
columns_answer = ['movieId', 'averageRating', 'ratingCounts']
assert_equal(set(columns), set(columns_answer))

total_rows = !sqlite3 problem3.db "SELECT COUNT(*) FROM validRatings;"
assert_equal(total_rows.s, "821")

first_five = !sqlite3 problem3.db "SELECT * FROM validRatings LIMIT 5;"
first_five_answer = ['1|3.87246963562753|247',
                     '2|3.4018691588785|107',
                     '3|3.16101694915254|59',
                     '5|3.26785714285714|56',
                     '6|3.88461538461538|104']
assert_equal(set(first_five), set(first_five_answer))

counts = !sqlite3 problem3.db "SELECT ratingCounts FROM validRatings WHERE movieId=497;"
assert_equal(counts.s, "60")

### 2. Join validRatings and movies into a new table


In [9]:
%%writefile join.sql
DROP TABLE IF EXISTS movieRatings;
CREATE TABLE movieRatings AS
  SELECT V.averageRating, M.title, M.genres
    FROM validRatings AS V, movies AS M
    WHERE V.movieId=M.movieId;

Overwriting join.sql


In [10]:
!sqlite3 problem3.db < join.sql

In [18]:
!sqlite3 problem3.db "SELECT * FROM movieRatings LIMIT 5;"

3.87246963562753|Toy Story (1995)|Adventure|Animation|Children|Comedy|Fantasy
3.4018691588785|Jumanji (1995)|Adventure|Children|Fantasy
3.16101694915254|Grumpier Old Men (1995)|Comedy|Romance
3.26785714285714|Father of the Bride Part II (1995)|Comedy
3.88461538461538|Heat (1995)|Action|Crime|Thriller


In [19]:
# sql assertion tests

movieRatings_exists = !sqlite3 problem3.db "SELECT name FROM sqlite_master WHERE type='table' and name='movieRatings';"
assert_equal(movieRatings_exists.s, "movieRatings", msg="There's no table named \"movieRatings\" in the database.")

movieRatings_info = !sqlite3 problem3.db "PRAGMA table_info(movieRatings);"
movieRatings_columns = [t.split("|")[1] for t in movieRatings_info]
movieRatings_columns_answer = ['averageRating', 'title', 'genres']
assert_equal(set(movieRatings_columns), set(movieRatings_columns_answer))

movieRatings_total_rows = !sqlite3 problem3.db "SELECT COUNT(*) FROM movieRatings;"
assert_equal(movieRatings_total_rows.s, "821")

movieRatings_first_five = !sqlite3 problem3.db "SELECT * FROM movieRatings LIMIT 5;"
movieRatings_first_five_answer = ['3.87246963562753|Toy Story (1995)|Adventure|Animation|Children|Comedy|Fantasy',
                                  '3.4018691588785|Jumanji (1995)|Adventure|Children|Fantasy',
                                  '3.16101694915254|Grumpier Old Men (1995)|Comedy|Romance',
                                  '3.26785714285714|Father of the Bride Part II (1995)|Comedy',
                                  '3.88461538461538|Heat (1995)|Action|Crime|Thriller']
assert_equal(set(movieRatings_first_five), set(movieRatings_first_five_answer))

rating = !sqlite3 problem3.db "SELECT averageRating FROM movieRatings WHERE title='Interstellar (2014)';"
assert_equal(rating.s, "4.15116279069767")


### 3. Find the best horror movies

In [25]:
%%writefile best_horror_movies.sql
SELECT * FROM movieRatings
WHERE averageRating>4.0 AND genres LIKE '%Horror%'
ORDER BY averageRating DESC;

Overwriting best_horror_movies.sql


In [26]:
!sqlite3 problem3.db < best_horror_movies.sql

4.25324675324675|Psycho (1960)|Crime|Horror
4.13815789473684|Silence of the Lambs, The (1991)|Crime|Horror|Thriller
4.02970297029703|Shining, The (1980)|Horror
4.01813471502591|Sixth Sense, The (1999)|Drama|Horror|Mystery


### 4. Find the most popular movie

In [28]:
%%writefile most_popular.sql
SELECT MAX(V.ratingCounts),M.title, M.genres 
FROM validRatings AS V, movies AS M
WHERE V.movieId=M.movieId;

Writing most_popular.sql


In [29]:
!sqlite3 problem3.db < most_popular.sql

341|Forrest Gump (1994)|Comedy|Drama|Romance|War


In [30]:
#sql assertion tests
most_popular_query = !sqlite3 problem3.db < most_popular.sql
most_popular_query_answer = "341|Forrest Gump (1994)|Comedy|Drama|Romance|War"
assert_equal(most_popular_query.s, most_popular_query_answer)

In [31]:
!rm problem3.db movies.sql ratings.sql best_horror_movies.sql \
 most_popular.sql movies.csv ratings.csv